In [10]:
import requests
import json
import csv,os,sys
from datetime import datetime
from bs4 import BeautifulSoup


#content = requests.get(url)
#page = BeautifulSoup(open('Coronavirus (COVID19) _ JAMA Network.html'), "html.parser")

jamapage = "https://jamanetwork.com/collections/46099/coronavirus-covid19?appId=scweb&fl_ContentType=Article&fl_Categories=Coronavirus+(COVID19)"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0",
    "Accept-Encoding": "*",
    "Connection": "keep-alive"
}
htmlpage = requests.get(jamapage, headers=headers)

page = BeautifulSoup(htmlpage.text, "html.parser")



In [11]:
def FormatAbstract (AB):

    Abstract = ''
    tempAbs = AB.split(' ')
    lastN=0
    newabs = []
    for N in range(30, len(tempAbs)+30, 30) :
        newabs.append(' '.join(tempAbs[lastN:N]))
        lastN= N

    Abstract = '\n'.join(newabs)

    return(Abstract)

In [12]:


txt = []
link = []
ID = []
AB = []
Dates = []
Type = []

for entry in page.find_all("a", attrs={"class": "article--title d-b mb05"}):
    txt.append(entry.text)
    link.append(entry.get('href'))
    ID.append(entry.get('href').split('/')[-1])
    
for abstract in page.find_all("div", attrs={"class": "article--excerpt"}): 
    if abstract.p != None:
        AB.append(FormatAbstract(abstract.p.text))
    else:
        AB.append('NA')
        
for Date in page.find_all("div", attrs={"class": "article--date meta-item no-wrap"}): 
    D = datetime.strptime(Date.text, '%B %d, %Y')
    Dates.append(D.strftime("%Y%m%d"))

for typemeta in page.find_all("div", attrs={"class": "article--type meta-item"}): 
    Type.append(typemeta.text)

In [13]:
json_file = '../jsonfiles/jama.json'
dict_file = '../jsonfiles/jama_dict_file.json'


#read the stored dictionary file (dict_file) or create a new blank dictionary
if os.path.isfile(dict_file):
    jsondict = json.load(open(dict_file))
else:
    jsondict = {}    
    
    
for N in range(0, len(ID)) :
    if ID[N] not in jsondict.keys():
        jsondict[ID[N]] = {'ID' : ID[N] , 
                          'txt' : txt[N],
                          'AB' : AB[N],
                          'link' : link[N],
                           'Dates' : Dates[N],
                           'Type' : Type[N]
                          }



#Write the main json file
with open(json_file , 'w') as fp:   
    Output=[]
    for Key,Item in jsondict.items():
        Output.append(Item)
    json.dump(Output, fp)

#Write the exact dict as json file (easily read by the script)
with open(dict_file , 'w') as fp:   
    json.dump(jsondict, fp)

In [14]:
def MakeTemplate():
    headerslist = []
    for H in ('ID' ,'txt' ,'AB' ,'link' ,'Dates' ,'Type' ):

        print(H)
        headers = '=ImportJSON("https://raw.githubusercontent.com/tofaquih/coronaPubGet/master/jsonfiles/jama.json", "/{}", "noInherit,noTruncate",$A$1)'.format(H)
        headerslist.append(headers)

    headerslist
    with open('../templates/JAMA_template.csv' ,'w'  , newline='' ) as fp:
        W = csv.writer(fp, delimiter=';')
        W.writerow(headerslist)
        

In [15]:
MakeTemplate()

ID
txt
AB
link
Dates
Type
